## Задание
- спарсить своего исполнителя
- добавить в бд поле с датой записи
- добавить имена артистов и ссылки на них 
- сделать это все с другим сайтом

### Параметры
- Сайт: https://www.songtexte.com
- Исполнитель: Lauren Daigle

# Libs

In [14]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
from datetime import datetime

In [3]:
url = 'https://www.songtexte.com/search?q=Lauren+Daigle&c=songs'
songs_list = requests.get(url)
songs_list.status_code

200

In [4]:
urls = ["https://www.songtexte.com/search?q=Lauren+Daigle&c=songs", "https://www.songtexte.com/?wicket:interface=:1:1:::",
       "https://www.songtexte.com/?wicket:interface=:1:2:::"]

songs_lists = []

for url in urls:
    songs_list = requests.get(url)
    
    songs_soup = BeautifulSoup(songs_list.content, "html.parser")
    for a in songs_soup.find_all('a'):
        if str(a).find('songtext') != -1 and str(a).find('lauren-daigle') != -1:
            song_url = "https://www.songtexte.com/" + str(a['href'])
            songs_lists.append(song_url)

songs_lists = list(set(songs_lists))

In [5]:
song_name = songs_lists[0]

song_url = requests.get(song_name)
song_soup = BeautifulSoup(song_url.content, 'html.parser') 
song_text = str(song_soup.find(id='lyrics')).rstrip().lstrip()
song_text

'<div id="lyrics">When the best of me is barely breathin′<br/>\nWhen I\'m not somebody I believe in<br/>\nHold on to me<br/>\n<br/>\nWhen I miss the light the night has stolen<br/>\nWhen I′m slammin\' all the doors You\'ve opened<br/>\nHold on to me<br/>\nHold on to me<br/>\n<br/>\nHold on to me when it′s too dark to see You<br/>\nWhen I am sure I have reached the end<br/>\nHold on to me when I forget I need You<br/>\nWhen I let go, hold me again<br/>\n<br/>\n<div class="hidden-print prmtnKeepHeight" id="npm_6670665"></div>\n<script type="text/javascript">\n<!--/*--><![CDATA[/*><!--*/\nADNPM.cmd.push(function(){ADNPM.loadAd(6670665)});\n/*-->]]>*/\n</script>\n<br/>\nWhen I don′t feel like I\'m worth defending<br/>\nWhen I′m tired of all my pretending<br/>\nHold on to me<br/>\n<br/>\nWhen I start to break in desperation<br/>\nUnderneath the weight of expectation<br/>\nHold on to me<br/>\nHold on to me<br/>\n<br/>\nHold on to me when it\'s too dark to see You (I′ll hold on)<br/>\nWhen I 

In [6]:
def replace_all(text, to_replace):
    for i in to_replace:
        text = text.replace(i, ' ')
    return text

In [8]:
parsed_songs = {} # словарик "url" :'текст песни'
n = 0 #счетчик


for song_name in songs_lists:
    song_url = requests.get(song_name)
    song_soup = BeautifulSoup(song_url.content, "html.parser")
    song_text = str(song_soup.find(id='lyrics'))
    
    song_text = replace_all(song_text, [
        '<','>', 'br/', '\n', 'div ', 
        'id="lyrics"', 
        'class="hidden-print prmtnKeepHeight"', 'id="npm_6670665"', '/', 'script type="text/javascript"','!--/*--','![CDATA[/*', 
        '/*-- ]] */  /script])',
        'script type="text javascript"',
        '!-- *--  ![CDATA[ *  !--*',
        'ADNPM.cmd.push(function(){ADNPM.loadAd(6670665)});',
        '*-- ]] *])', '*-- ]] *', '  '])
        
    song_text = " ".join(song_text.split())

    parsed_songs[song_name] = song_text
    n += 1
    print('Parsed', n, '/', len(songs_lists))
    time.sleep(1)

Parsed 1 / 25
Parsed 2 / 25
Parsed 3 / 25
Parsed 4 / 25
Parsed 5 / 25
Parsed 6 / 25
Parsed 7 / 25
Parsed 8 / 25
Parsed 9 / 25
Parsed 10 / 25
Parsed 11 / 25
Parsed 12 / 25
Parsed 13 / 25
Parsed 14 / 25
Parsed 15 / 25
Parsed 16 / 25
Parsed 17 / 25
Parsed 18 / 25
Parsed 19 / 25
Parsed 20 / 25
Parsed 21 / 25
Parsed 22 / 25
Parsed 23 / 25
Parsed 24 / 25
Parsed 25 / 25


In [11]:
cur.execute("DROP TABLE songs_of_Lauren_Daigle")

In [12]:
con = sqlite3.connect("songs_of_Lauren_Daigle.db") #создали локальную БД

cur = con.cursor() 
cur.execute("CREATE TABLE songs_of_Lauren_Daigle (author, author_url, text, text_url, time)") 

In [16]:
url = 'https://www.songtexte.com/artist/lauren-daigle-3bd9e03c.html'
sql_insert = "INSERT INTO songs_of_Lauren_Daigle (author, author_url, text, text_url, time) VALUES (?,?,?,?,?)"
for k, v in parsed_songs.items():
    res = cur.execute(sql_insert,('Lauren_Daigle', url, v, k, datetime.now()))
    con.commit()

res = cur.execute("Select * from songs_of_Lauren_Daigle")
result = res.fetchall()
len(result)

25

In [17]:
result[0]

('Lauren_Daigle',
 'https://www.songtexte.com/artist/lauren-daigle-3bd9e03c.html',
 "When the best of me is barely breathin′ When I'm not somebody I believe in Hold on to me When I miss the light the night has stolen When I′m slammin' all the doors You've opened Hold on to me Hold on to me Hold on to me when it′s too dark to see You When I am sure I have reached the end Hold on to me when I forget I need You When I let go, hold me again script When I don′t feel like I'm worth defending When I′m tired of all my pretending Hold on to me When I start to break in desperation Underneath the weight of expectation Hold on to me Hold on to me Hold on to me when it's too dark to see You (I′ll hold on) When I am sure I have reached the end Hold on to me when I forget I need You (I'll hold on) When I let go, hold me again I could rest here in Your arms forever ′Cause I know nobody loves me better Hold on to me Hold on to me",
 'https://www.songtexte.com/songtext/lauren-daigle/hold-on-to-me-g4bbc8

## ЗАДАЧА 2
- Спарсить исполнителей и ссылки на них